In this notebook, we show a simple implementation of BottleNet paper, in which we add a non-differentiable layer to a neural network and try to implement its custom gradient.

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torchvision import transforms

from io import BytesIO
from PIL import Image

We can define custom functions with user-defined derivatives in torch. In the forward pass, we compute the JPEG compressed/decompressed values on the feature tensor. In the backward pass, we approximate the derivative of the JPEG compressor/decompressor pair by the identity function, which means the output gradients will back-propagate to the previous layer without any change. 

In [2]:
class BottleneckUnit(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        result = torch.zeros_like(x)
        batch_size, channels, w, h = x.shape
        outputIoStream = BytesIO()
        image = transforms.ToPILImage()(x.view(batch_size*channels,w*h)) #batch_size*channels, w*h))
        image.save(outputIoStream, "JPEG", quality=90)
        outputIoStream.seek(0)
        decompressed = Image.open(outputIoStream)
        result = transforms.ToTensor()(decompressed)
        result = result.view(batch_size, channels, w, h)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.conv3 = nn.Conv2d(64, 64, 3, 1)
        self.dropout = nn.Dropout2d(0.1)
        self.fc1 = nn.Linear(6400, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout(x)
        x = BottleneckUnit.apply(x) #this is where we add our JPEG compressor/decompressor
        x = self.conv3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

The rest of the code is a typical training loop and optimizers.

In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 30 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

In [5]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
use_cuda = torch.cuda.is_available()
torch.manual_seed(0)
device = torch.device("cuda" if use_cuda else "cpu")
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                   transforms.ToTensor(),
                   transforms.Normalize((0.1307,), (0.3081,))
               ])),
    batch_size=128, shuffle=False, num_workers=2, pin_memory=True)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.1)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, 3):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302764
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.309572
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.842997
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.893695
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.688249
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.854157
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.591457
Train Epoch: 1 [26880/60000 (45%)]	Loss: 0.466911
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.660444
Train Epoch: 1 [34560/60000 (58%)]	Loss: 0.444245
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.581069
Train Epoch: 1 [42240/60000 (70%)]	Loss: 0.335881
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.464150
Train Epoch: 1 [49920/60000 (83%)]	Loss: 0.317720
Train Epoch: 1 [53760/60000 (90%)]	Loss: 0.400302
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.759805

Test set: Average loss: 0.4868, Accuracy: 8641/10000 (86%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.538097
Train Epoch: 2 [3840/60000 (6%)]	Loss: 0.574778
Train Epoch: 2 [7680/60000 (13%)]	Loss: 0.388432
Train